<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>


---


<p align="center"><h1 align="center">Used Car Sales Price Prediction Competition

##### <p align="center">*Note: Dataset adapted from N. Birla's public dataset ['Vehicle Dataset'](https://www.kaggle.com/nehalbirla/vehicle-dataset-from-cardekho)* 
---
Let's share our models to a centralized leaderboard, so that we can collaborate and learn from the model experimentation process...

**Instructions:**
1.   Get data in and set up X_train / X_test / y_train
2.   Preprocess data  with Sklearn Column Transformer/ Write and Save Preprocessor function
3. Fit model on preprocessed data and save preprocessor function and model 
4. Generate predictions from X_test data and submit model to competition
5. Repeat submission process to improve place on leaderboard



## 1. Get data in and set up X_train, X_test, y_train objects

In [ ]:
#install aimodelshare library
! pip install aimodelshare --upgrade

In [2]:
# Get competition data
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/used_car_competition_data-repository:latest') 


Data downloaded successfully.


In [3]:
# Separate data into X_train, y_train, and X_test
import pandas as pd

X_train = pd.read_csv("/content/used_car_competition_data/training_data.csv")
y_train = X_train['selling_price']
X_train.drop(['selling_price'], axis=1, inplace=True)

X_test=pd.read_csv("/content/used_car_competition_data/test_data.csv")

X_train.head()

,year,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats
0,2011,84000,Diesel,Individual,Manual,Second Owner,20.00,1399.0,68.00,5.0
1,2014,74537,Petrol,Individual,Manual,Second Owner,20.51,998.0,67.04,5.0
2,2008,110000,Petrol,Individual,Manual,Second Owner,17.70,1497.0,78.00,5.0
3,2019,5000,Petrol,Individual,Automatic,First Owner,21.70,998.0,67.00,5.0
4,2017,45000,Diesel,Dealer,Automatic,First Owner,19.33,1999.0,177.00,5.0


##2.   Preprocess data using Sklearn Column Transformer / Write and Save Preprocessor function


In [4]:
# In this case we use Sklearn's Column transformer in our preprocessor function

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#Preprocess data using sklearn's Column Transformer approach

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_features = ['km_driven', 'mileage', 'engine', 'max_power', 'year']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), #'imputer' names the step
    ('scaler', StandardScaler())])

categorical_features = ['fuel', 'seller_type', 'transmission', 'owner', 'seats']

# Replacing missing values with Modal value and then one-hot encoding.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Final preprocessor object set up with ColumnTransformer...

preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# fit preprocessor to your data
preprocess = preprocess.fit(X_train)

In [5]:
# Write function to transform data with preprocessor 
# In this case we use sklearn's Column transformer in our preprocessor function

def preprocessor(data):
    preprocessed_data=preprocess.transform(data)
    return preprocessed_data

In [6]:
# check shape of X data 
preprocessor(X_train).shape

(6502, 27)

##3. Fit model on preprocessed data and save preprocessor function and model 


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import keras

feature_count=preprocessor(X_train).shape[1] #count features in input data

model = Sequential()
model.add(Dense(64, input_dim=feature_count, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))

model.add(Dense(1, kernel_initializer='normal')) 
                                            
# Compile model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# Fitting the NN to the Training set
model.fit(preprocessor(X_train), y_train, 
               batch_size = 60, 
               epochs = 50, validation_split=0.35)  

#### Save preprocessor function to local "preprocessor.zip" file

In [8]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

Your preprocessor is now saved to 'preprocessor.zip'


#### Save model to local ".onnx" file

In [9]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

# Check how many preprocessed input features there are
from skl2onnx.common.data_types import FloatTensorType

onnx_model = model_to_onnx(model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## 4. Generate predictions from X_test data and submit model to competition


In [10]:
#Set credentials using modelshare.org username/password

from aimodelshare.aws import set_credentials
    
apiurl="https://tv6fawq47b.execute-api.us-east-1.amazonaws.com/prod/m" #This is the unique rest api that powers this Used Car Sales Prediction Playground

set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [11]:
#Instantiate Competition
import aimodelshare as ai
mycompetition= ai.Competition(apiurl)

In [12]:
#Submit Model 1: 

#-- Generate predicted values (a list of predicted car prices) (Model 1)
predicted_values = model.predict(preprocessor(X_test))

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=predicted_values)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 7

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1670


In [13]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

,mse,rmse,mae,r2,ml_framework,transfer_learning,deep_learning,model_type,depth,num_params,dense_layers,relu_act,loss,optimizer,memory_size,username,version
0,7671996.51,2769.84,1677.97,0.94,sklearn,False,False,RandomForestRegressor,nan,nan,nan,nan,nan,nan,nan,AIModelShare,5
1,7949311.04,2819.45,1598.95,0.94,keras,False,True,Sequential,4.000000,10177.000000,4.000000,3.000000,str,Adam,1302256.000000,AIModelShare,7
2,15060949.12,3880.84,2430.51,0.88,sklearn,False,False,GradientBoostingRegressor,nan,nan,nan,nan,nan,nan,nan,AIModelShare,6
3,15516590.45,3939.11,2493.11,0.88,sklearn,False,False,RandomForestRegressor,nan,nan,nan,nan,nan,nan,nan,AIModelShare,1
4,15516590.45,3939.11,2493.11,0.88,sklearn,False,False,RandomForestRegressor,nan,nan,nan,nan,nan,nan,nan,AIModelShare,3
5,22666752.77,4760.96,3282.78,0.82,sklearn,False,False,RandomForestRegressor,nan,nan,nan,nan,nan,nan,nan,AIModelShare,2
6,22666752.77,4760.96,3282.78,0.82,sklearn,False,False,RandomForestRegressor,nan,nan,nan,nan,nan,nan,nan,AIModelShare,4


## 5. Repeat submission process to improve place on leaderboard


In [ ]:
# Create model 2  -- Model with Dropout regularization
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout,BatchNormalization

feature_count=preprocessor(X_train).shape[1] #count features in input data

model_2 = Sequential()
model_2.add(Dense(64, input_dim=feature_count))
model_2.add(BatchNormalization())
model_2.add(Activation('relu'))
model_2.add(Dense(64))
model_2.add(BatchNormalization())
model_2.add(Activation('relu'))
model_2.add(Dense(64))
model_2.add(BatchNormalization())
model_2.add(Activation('relu'))
model_2.add(Dense(1, kernel_initializer='normal')) 
                                            
# Compile model
model_2.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# Fitting the NN to the Training set
model_2.fit(preprocessor(X_train), y_train, 
               batch_size = 60, 
               epochs = 60, validation_split=0.35)
                                        

In [15]:
# Save Model 2 to .onnx file

onnx_model = model_to_onnx(model_2, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

# Save model to local .onnx file
with open("model_2.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString()) 

In [16]:
# Submit Model 2

#-- Generate predicted y values (Model 2)
prediction_labels = model_2.predict(preprocessor(X_test))

# Submit Model 2 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model_2.onnx",
                                 prediction_submission=prediction_labels,
                                 preprocessor_filepath="preprocessor.zip")

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 8

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1670


In [17]:
# Compare two or more models
data=mycompetition.compare_models([7, 8], verbose=1)
mycompetition.stylize_compare(data)

,Model_7_Layer,Model_7_Shape,Model_7_Params,Model_8_Layer,Model_8_Shape,Model_8_Params
0,Dense,"[None, 64]",1792.000000,Dense,"[None, 64]",1792
1,Dense,"[None, 64]",4160.000000,BatchNormalization,"[None, 64]",256
2,Dense,"[None, 64]",4160.000000,Activation,"[None, 64]",0
3,Dense,"[None, 1]",65.000000,Dense,"[None, 64]",4160
4,None,None,nan,BatchNormalization,"[None, 64]",256
5,None,None,nan,Activation,"[None, 64]",0
6,None,None,nan,Dense,"[None, 64]",4160
7,None,None,nan,BatchNormalization,"[None, 64]",256
8,None,None,nan,Activation,"[None, 64]",0
9,None,None,nan,Dense,"[None, 1]",65


## Optional: Tune model within range of hyperparameters with Keras Tuner

*Simple example shown below. Consult [documentation](https://keras.io/guides/keras_tuner/getting_started/) to see full functionality.*

In [ ]:
! pip install keras_tuner

In [19]:
#Separate validation data 
from sklearn.model_selection import train_test_split
x_train_split, x_val, y_train_split, y_val = train_test_split(
     X_train, y_train, test_size=0.2, random_state=42)

In [20]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from keras.regularizers import l1, l2, l1_l2
import keras_tuner as kt

feature_count=preprocessor(X_train).shape[1] #count features in input data

#Define model structure & parameter search space with function
def build_model(hp):
    model = keras.Sequential()
    model.add(Dense(64, input_dim=feature_count, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dense(units=hp.Int("units", min_value=32, max_value=512, step=32), #range 32-512 inclusive, minimum step between tested values is 32
                    activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dense(1, kernel_initializer='normal')) 
    model.compile(
        loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
    return model                                          

#initialize the tuner (which will search through parameters)
tuner = kt.RandomSearch(
    hypermodel=build_model, 
    objective="mean_squared_error", # objective to optimize
    max_trials=3, #max number of trials to run during search
    executions_per_trial=3, #higher number reduces variance of results; guages model performance more accurately 
    overwrite=True,
    directory="tuning_model",
    project_name="tuning_units",
)

tuner.search(preprocessor(x_train_split), y_train_split, epochs=2, validation_data=(preprocessor(x_val), y_val))

Trial 3 Complete [00h 00m 11s]
mean_squared_error: 63057820.0

Best mean_squared_error So Far: 42746766.666666664
Total elapsed time: 00h 00m 42s


In [21]:
# Build model with best hyperparameters

# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
tuned_model = build_model(best_hps[0])
# Fit with the entire dataset.
tuned_model.fit(x=preprocessor(X_train), y=y_train, epochs=5)

Epoch 1/5
204/204 [==============================] - 1s 3ms/step - loss: 132252280.0000 - mean_squared_error: 132252272.0000
Epoch 2/5
204/204 [==============================] - 1s 3ms/step - loss: 33343672.0000 - mean_squared_error: 33343654.0000
Epoch 3/5
204/204 [==============================] - 1s 3ms/step - loss: 23279780.0000 - mean_squared_error: 23279766.0000
Epoch 4/5
204/204 [==============================] - 1s 3ms/step - loss: 17887898.0000 - mean_squared_error: 17887880.0000
Epoch 5/5
204/204 [==============================] - 1s 3ms/step - loss: 14445863.0000 - mean_squared_error: 14445840.0000


In [22]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(tuned_model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("tuned_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [23]:
#Submit Model 3: 

#-- Generate predicted values
prediction_labels = tuned_model.predict(preprocessor(X_test))

# Submit Model 3 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "tuned_model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 9

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1670


In [24]:
# Get leaderboard

data = mycompetition.get_leaderboard()
mycompetition.stylize_leaderboard(data)

,mse,rmse,mae,r2,ml_framework,transfer_learning,deep_learning,model_type,depth,num_params,batchnormalization_layers,dense_layers,relu_act,loss,optimizer,memory_size,username,version
0,7671996.51,2769.84,1677.97,0.94,sklearn,False,False,RandomForestRegressor,nan,nan,nan,nan,nan,nan,nan,nan,AIModelShare,5
1,7949311.04,2819.45,1598.95,0.94,keras,False,True,Sequential,4.000000,10177.000000,nan,4.000000,3.000000,str,Adam,1302256.000000,AIModelShare,7
2,11541482.51,3397.28,1894.00,0.91,keras,False,True,Sequential,4.000000,35521.000000,nan,4.000000,3.000000,str,Adam,1583768.000000,AIModelShare,9
3,15060949.12,3880.84,2430.51,0.88,sklearn,False,False,GradientBoostingRegressor,nan,nan,nan,nan,nan,nan,nan,nan,AIModelShare,6
4,15516590.45,3939.11,2493.11,0.88,sklearn,False,False,RandomForestRegressor,nan,nan,nan,nan,nan,nan,nan,nan,AIModelShare,1
5,15516590.45,3939.11,2493.11,0.88,sklearn,False,False,RandomForestRegressor,nan,nan,nan,nan,nan,nan,nan,nan,AIModelShare,3
6,22666752.77,4760.96,3282.78,0.82,sklearn,False,False,RandomForestRegressor,nan,nan,nan,nan,nan,nan,nan,nan,AIModelShare,2
7,22666752.77,4760.96,3282.78,0.82,sklearn,False,False,RandomForestRegressor,nan,nan,nan,nan,nan,nan,nan,nan,AIModelShare,4
8,128996657.60,11357.67,6997.92,-0.02,keras,False,True,Sequential,7.000000,10945.000000,3.000000,4.000000,3.000000,str,Adam,2921072.000000,AIModelShare,8


In [28]:
# Compare two or more models
data=mycompetition.compare_models([7, 8, 9], verbose=1)
mycompetition.stylize_compare(data)

,Model_7_Layer,Model_7_Shape,Model_7_Params,Model_8_Layer,Model_8_Shape,Model_8_Params,Model_9_Layer,Model_9_Shape,Model_9_Params
0,Dense,"[None, 64]",1792.000000,Dense,"[None, 64]",1792,Dense,"[None, 64]",1792.000000
1,Dense,"[None, 64]",4160.000000,BatchNormalization,"[None, 64]",256,Dense,"[None, 64]",4160.000000
2,Dense,"[None, 64]",4160.000000,Activation,"[None, 64]",0,Dense,"[None, 448]",29120.000000
3,Dense,"[None, 1]",65.000000,Dense,"[None, 64]",4160,Dense,"[None, 1]",449.000000
4,None,None,nan,BatchNormalization,"[None, 64]",256,None,None,nan
5,None,None,nan,Activation,"[None, 64]",0,None,None,nan
6,None,None,nan,Dense,"[None, 64]",4160,None,None,nan
7,None,None,nan,BatchNormalization,"[None, 64]",256,None,None,nan
8,None,None,nan,Activation,"[None, 64]",0,None,None,nan
9,None,None,nan,Dense,"[None, 1]",65,None,None,nan
